#### [1. Import Packages](#1)
#### [2. Engineer Functions](#2)
- ##### [2.1 Polymarket Functions](#2_1)
- ##### [2.2 Kalshi Functions](#2_2)
#### [3. Arbitrage](#3)
- ##### [3.1 Collect Polymarket Data](#3_1)
    - ##### [3.2.1. Collect Polymarket Orderbook](#3_1_1)
- ##### [3.2 Collect Kalshi Data](#3_2)
    - ##### [3.2.1. Collect Kalshi Orderbook](#3_2_1)
- ##### [3.3 Compare PM and Kalshi Data for One Market](#3_3)
    - ##### [3.3.1 Match Event Tickers and Slugs](#3_3_1)
    - ##### [3.3.1 Trade on Polymarket](#3_3_1)
    - ##### [3.3.2 Trade on Kalshi](#3_3_2)
- ##### [3.4 Identify Arbitrage and Trade](#3_4)
    - ##### [3.4.1 Specify Tickers and Trade](#3_4_1)

## 1. Import Packages <a id='1'></a>

In [107]:
import pandas as pd
import numpy as np
import requests
import json
import time
import math
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, OrderType
from py_clob_client.order_builder.constants import BUY, SELL

import os, uuid, base64, requests
from email.utils import parsedate_to_datetime
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding




## 2. Engineer Functions <a id='2'></a>

### 2.1 Polymarket Functions <a id='2_1'></a>

**discover_games_slugs_by_date(start_date, end_date, tag_slug)**:
- Discover slugs using date-filtered events API
- Default tag_slug = "MLB"

**get_polymarket_market_data(slug)**:
- Get the Market data for that specific event with its slug

**extract_game_info(market_data, slug)**:
- Extract comprehensive game information from market data using slug

**get_polymarket_games_markets_df(tag_slug)**:
- Get current markets using the user's proven discovery method, default is tag_slug = "MLB"

**get_polymarket_orderbook(market_id)**:
- Get the entire orderbook from a single polymarket market
- Output has orderbook for "yes" and "no" price for that specific event ("Will Atlanta Braves win tonight?")
- Returns both bid and ask price

**discover_future_slugs(tag_slug)**:
- Get future slugs. Default to search MLB future events.

**get_polymarket_future_markets_df(tag_slug)**:
- Retrieve data for future market, like the NBA championship

**get_polymarket_orderbook_array(ticker_a, ticker_b)**:
- Return the bid and ask prices for a single market.
- We return the orderbooks from both sides, since orderbooks are mirrored in multi outcome event.

**trade_limit_order_polymarket(pm_ticker_a, price, size, side=1, order_type=0)**:
- Place a limit order with a spcific price, size, side (1 == buy else is sell), order_type(0 == GTC else IOC)

**pm_to_kalshi(pm_slug: str)**:
- Convert PM slug to kalshi ticker

In [108]:
def discover_games_slugs_by_date(start_date, end_date, tag_slug="mlb"):
    """
    Discover MLB game slugs using date-filtered events API
    Based on: https://gamma-api.polymarket.com/events?tag_slug=mlb&active=true&start_date_min=DATE&start_date_max=NEXT
    """
    print(f"🔍 Discovering slugs from {start_date} to {end_date}...")
    
    url = "https://gamma-api.polymarket.com/events"
    params = {
        'tag_slug': tag_slug,
        'active': 'true',
        'start_date_min': f'{start_date}T00:00:00Z',
        'start_date_max': f'{end_date}T00:00:00Z'
    }
    
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        # if the request is successful, we get the data
        if response.status_code == 200: # 200 means success, 400 means error, 500 means server error
            events_data = response.json()
            if events_data and isinstance(events_data, list): # if the output is a list
                slugs = [event.get('slug') for event in events_data if event.get('slug')] # get the slug for each event
                print(f"   ✅ Found {len(slugs)} event slugs")
                return slugs
            else:
                print(f"   ❌ No events data returned")
                return []
        else:
            print(f"   ❌ API Error: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"   ❌ Exception: {str(e)}")
        return []

def get_polymarket_market_data(slug):
    """
    Get market data for a specific game slug
    Based on: https://gamma-api.polymarket.com/markets?active=true&slug=SLUG
    """
    url = "https://gamma-api.polymarket.com/markets"
    params = {
        'active': 'true', # only get active markets
        'slug': slug 
    }
    
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        
        # if the request is successful, we get the data
        if response.status_code == 200: # 200 means success, 400 means error, 500 means server error
            data = response.json() # convert the response to a JSON object
            return data if data else [] # return the data if it exists, otherwise return an empty list
        else:
            return []
            
    except Exception as e:
        return []

def extract_game_info(market_data, slug):
    """
    Extract comprehensive game information from market data
    """
    if not market_data or len(market_data) == 0:
        print(f"⚠️ No market data found for {slug}")
        return None
    
    market = market_data[0]  # Take first market
    
    # Extract comprehensive market info
    game_info = {
        # Basic identifiers
        'slug': slug,
        'market_id': market.get('id', 'Unknown'),
        'condition_id': market.get('conditionId', 'Unknown'),
        'question_id': market.get('questionID', 'Unknown'),
        
        # Game info
        'title': market.get('question', 'Unknown Game'),
        'game_time': market.get('gameStartTime', 'Unknown'),
        'description': market.get('description', ''),
        'resolution_source': market.get('resolutionSource', ''),
        
        # Market status
        'active': market.get('active', False),
        'closed': market.get('closed', True),
        'accepting_orders': market.get('acceptingOrders', False),
        'accepting_orders_timestamp': market.get('acceptingOrdersTimestamp', ''),
        'new': market.get('new', False),
        'archived': market.get('archived', False),
        'restricted': market.get('restricted', False),
        
        # Trading info
        'market_maker_address': market.get('marketMakerAddress', ''),
        'enable_order_book': market.get('enableOrderBook', False),
        'order_min_size': market.get('orderMinSize', 0),
        'order_price_min_tick_size': market.get('orderPriceMinTickSize', 0),
        'clob_token_ids': market.get('clobTokenIds', ''),
        
        # Pricing
        'volume': float(market.get('volume', 0)),
        'liquidity': float(market.get('liquidity', 0)),
        'volume_24hr': float(market.get('volume24hr', 0)),
        'volume_1wk': float(market.get('volume1wk', 0)),
        'volume_1mo': float(market.get('volume1mo', 0)),
        'volume_1yr': float(market.get('volume1yr', 0)),
        'best_bid': float(market.get('bestBid', 0)),
        'best_ask': float(market.get('bestAsk', 0)),
        'spread': float(market.get('spread', 0)),
        'last_trade_price': float(market.get('lastTradePrice', 0)),
        'one_day_price_change': float(market.get('oneDayPriceChange', 0)),
        'one_hour_price_change': float(market.get('oneHourPriceChange', 0)),
        'competitive': float(market.get('competitive', 0)),
        
        # Dates and timing
        'start_date': market.get('startDate', ''),
        'end_date': market.get('endDate', ''),
        'created_at': market.get('createdAt', ''),
        'updated_at': market.get('updatedAt', ''),
        
        # Market structure
        'fee': market.get('fee', ''),
        'fpmm_live': market.get('fpmmLive', False),
        'neg_risk': market.get('negRisk', False),
        'rfq_enabled': market.get('rfqEnabled', False),
        'approved': market.get('approved', False),
        'funded': market.get('funded', False),
        
        # Outcomes and prices (parsed)
        'outcomes': [],
        'prices': []
    }
    
    # Extract outcomes and prices using proven method
    try:
        if 'outcomes' in market and market['outcomes']: # if the outcomes are in the market
            game_info['outcomes'] = json.loads(market['outcomes']) # parse the outcomes
            
        if 'outcomePrices' in market and market['outcomePrices']: # if the outcome prices are in the market
            price_strings = json.loads(market['outcomePrices']) # parse the outcome prices
            game_info['prices'] = [float(p) for p in price_strings] # convert the outcome prices to floats
            
        # Parse CLOB token IDs if available 
        if 'clobTokenIds' in market and market['clobTokenIds']: # if the CLOB token IDs are in the market   
            try:
                game_info['clob_token_ids'] = json.loads(market['clobTokenIds']) # parse the CLOB token IDs
            except:
                game_info['clob_token_ids'] = market['clobTokenIds']  # Keep as string if parsing fails
                
    except Exception as e:
        print(f"⚠️ Price parsing error for {slug}: {e}")
    
    return game_info

def get_polymarket_games_markets_df(tag_slug="mlb"):
    """
    Get current MLB markets using the user's proven discovery method,
    and return results as a pandas DataFrame.
    """
    print("⚾ POLYMARKET MLB LIVE MARKETS (Date-Filtered Discovery)")
    print("=" * 60)
    print(f"🕐 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("🔗 Using date-filtered events API discovery")
    print()
    
    # Build the list of dates to check (today)
    today = datetime.now().date() - timedelta(days=1)
    dates_to_check = [today.strftime('%Y-%m-%d')]
    
    all_games = []
    seen_slugs = set()
    
    for date_str in dates_to_check:
        # compute next day
        next_date = (datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
        
        print(f"🔍 DISCOVERING EVENTS: {date_str} to {next_date}")
        discovered_slugs = discover_games_slugs_by_date(date_str, next_date, tag_slug)
        
        if not discovered_slugs:
            print(f"   ❌ No events found for {date_str}\n")
            continue
    
        for slug in discovered_slugs:
            if slug in seen_slugs:
                print(f"   ⏭️ Skipping duplicate: {slug}")
                continue
            seen_slugs.add(slug)
            print(f"   🔍 Getting market data for {slug}")
            mkt = get_polymarket_market_data(slug)
            info = extract_game_info(mkt, slug) if mkt else None
            if info:
                all_games.append(info)
    # Convert to DataFrame
    df = pd.DataFrame(all_games)    
    print("\n📊 FINAL SUMMARY")
    print("=" * 60)
    print(f"🎯 Total live games found: {len(df)}")
    return df

def discover_future_slugs(tag_slug ="mlb", limit=100, specific_event=None):
    """
    Discover MLB game slugs using date-filtered events API
    Based on: https://gamma-api.polymarket.com/events?tag_slug=mlb&active=true&start_date_min=DATE&start_date_max=NEXT
    """
    print(f"🔍 Discovering slugs for {tag_slug}")

    
    url = "https://gamma-api.polymarket.com/events"
    params = {
        'tag_slug': tag_slug,
        'limit': limit,
        'active': 'true',
        'closed': 'false',
        'archived': 'false'
    }
    
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        # if the request is successful, we get the data
        if response.status_code == 200: # 200 means success, 400 means error, 500 means server error
            events_data = response.json()
            if events_data and isinstance(events_data, list): # if the output is a list
                
                slugs = [event.get('slug') for event in events_data if event.get('slug')] # get the slug for each event
                if specific_event:
                    slugs = [i for i in slugs if specific_event.lower() in i.lower()]
                print(f"   ✅ Found {len(slugs)} event slugs")
                return slugs
            else:
                print(f"   ❌ No events data returned")
                return []
        else: 
            print(f"   ❌ API Error: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"   ❌ Exception: {str(e)}")
        return []
    
def get_polymarket_orderbook(token_id):
    """
    Get orderbook for a specific token_id.
    """
    url = "https://clob.polymarket.com/book"
    querystring = {"token_id":token_id} # token_id is the ID of the token you want to get the orderbook for
    response = requests.request("GET", url, params=querystring) # send a GET request to the URL with the token_id   
    if response.status_code == 200: # 200 means success, 400 means error, 500 means server error
        return response.json()
    else:
        return None
    
def get_polymarket_future_markets_df(tag_slug="mlb", specific_event='world-series-champion-2025'):
    """
    Get current MLB markets using the user's proven discovery method,
    and return results as a pandas DataFrame.
    """
    print("⚾ POLYMARKET MLB LIVE MARKETS (Date-Filtered Discovery)")
    print("=" * 60)
    print(f"🕐 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("🔗 Using date-filtered events API discovery")
    print()
    
    # Build the list of dates to check (today)

    all_games = []
    seen_slugs = set()
    
    print(f"🔍 DISCOVERING FUTURE EVENTS for {tag_slug}")
    discovered_slugs = discover_future_slugs(tag_slug=tag_slug, specific_event=specific_event)
    
    if discovered_slugs:
        for slug in discovered_slugs:
            if slug in seen_slugs:
                print(f"   ⏭️ Skipping duplicate: {slug}")
                continue
            seen_slugs.add(slug)
            print(f"   🔍 Getting market data for {slug}")
            mkt = get_polymarket_market_data(slug)
            info = extract_game_info(mkt, slug) if mkt else None
            if info:
                all_games.append(info)
    else:
        print(f"   ❌ No events found for {tag_slug}")
        return None
    # Convert to DataFrame
    df = pd.DataFrame(all_games)    
    print("\n📊 FINAL SUMMARY")
    print("=" * 60)
    print(f"🎯 Total live games found: {len(df)}")
    return df
    
def get_polymarket_orderbook_array_with_tickers(pm_ticker_a, pm_ticker_b, tick_size=0.01):
    """
    Get orderbook for a specific token_id.
    """
    # ------------------------------
    # Output from get_polymarket_orderbook() is a dictionary object.
    # ------------------------------
    start = time.time()
    pm_ticker_a_orderbook = get_polymarket_orderbook(pm_ticker_a) # Get json response for ticker_a's orderbook. 
    pm_ticker_b_orderbook = get_polymarket_orderbook(pm_ticker_b) # Get json response for ticker_b's orderbook. 
    # print(pm_ticker_a_orderbook)
    end = time.time()
    # print(f"Time taken to get orderbook: {end - start} seconds")

    # a list of dictionary, each dictionary contains 'price' and 'size'
    pm_bids_a = pm_ticker_a_orderbook['bids']
    pm_asks_a = pm_ticker_a_orderbook['asks']

    pm_bids_b = pm_ticker_b_orderbook['bids']
    pm_asks_b = pm_ticker_b_orderbook['asks']
    if tick_size == 0.01:
        start = time.time()
        # ------------------------------
        # ticker_a orderbook
        # ------------------------------
        # The calculation here can be done in a more efficient way.
        # We should adjust how we calculate price based on minimum tick size. 
        pm_asks_a_array = np.array([[np.int64(round(np.float64(ask['price']) * 100)), np.int64(np.float64(ask['size']))] for ask in pm_asks_a])
        pm_bids_a_array = np.array([[np.int64(round(np.float64(bid['price']) * 100)), np.int64(np.float64(bid['size']))] for bid in pm_bids_a])
        pm_bids_a_array = pm_bids_a_array[::-1]

        # ------------------------------
        # ticker_b orderbook
        # ------------------------------
        pm_asks_b_array = np.array([[np.int64(round(np.float64(ask['price']) * 100)), np.int64(np.float64(ask['size']))] for ask in pm_asks_b])
        pm_bids_b_array = np.array([[np.int64(round(np.float64(bid['price']) * 100)), np.int64(np.float64(bid['size']))] for bid in pm_bids_b])
        pm_bids_b_array = pm_bids_b_array[::-1]
        end = time.time()
        # print(f"Time taken to process numpy array: {end - start} seconds")
    else:
        return None, None, None, None

    return pm_asks_a_array, pm_bids_a_array, pm_asks_b_array, pm_bids_b_array

def pm_to_kalshi(pm_slug: str) -> str:
    """
    Convert a Polymarket slug (e.g. "mlb-ari-sd-2025-07-10")
    into a Kalshi ticker (e.g. "KXMLBGAME-25JUL10ARISD").
    """
    tag, t1, t2, yyyy, mm, dd = pm_slug.split('-')
    # build date object
    dt = datetime.strptime(f"{yyyy}-{mm}-{dd}", "%Y-%m-%d")
    date_part = dt.strftime("%y%b%d").upper()        # "25JUL10"
    return f"KX{tag.upper()}GAME-{date_part}{t1.upper()}{t2.upper()}"

def trade_limit_order_polymarket(pm_ticker_a, price, size, side=1, order_type=0):
    host: str = "https://clob.polymarket.com"
    key: str = os.getenv("POLYMARKET_KEY") #This is your Private Key. Export from reveal.polymarket.com or from your Web3 Application
    chain_id: int = 137 #No need to adjust this
    POLYMARKET_PROXY_ADDRESS: str = '0x3F7f6D11A8b4d504b6B0dd54e963F0cD51517720' #This is the address you deposit/send USDC to to FUND your Polymarket account.

    ### Initialization of a client using a Polymarket Proxy associated with an Email/Magic account. If you login with your email use this example.
    client = ClobClient(host, key=key, chain_id=chain_id, signature_type=1, funder=POLYMARKET_PROXY_ADDRESS)

    ## Create and sign a limit order buying 5 tokens for 0.010c each
    #Refer to the Markets API documentation to locate a tokenID: https://docs.polymarket.com/developers/gamma-markets-api/get-markets

    client.set_api_creds(client.create_or_derive_api_creds()) 
    
    side = BUY if side == 1 else SELL
    order_args = OrderArgs(
        price=price,
        size=size,
        side=side,
        token_id=pm_ticker_a, #Token ID you want to purchase goes here. 
    )
    signed_order = client.create_order(order_args)

    ## GTC(Good-Till-Cancelled) Order
    order_type = OrderType.GTC if order_type == 0 else OrderType.IOC
    resp = client.post_order(signed_order, order_type)
    print(resp)

In [109]:
def discover_future_slugs(tag_slug ="mlb", limit=100, specific_event=None):
    """
    Discover MLB game slugs using date-filtered events API
    Based on: https://gamma-api.polymarket.com/events?tag_slug=mlb&active=true&start_date_min=DATE&start_date_max=NEXT
    """
    print(f"🔍 Discovering slugs for {tag_slug}")

    
    url = "https://gamma-api.polymarket.com/events"
    params = {
        'tag_slug': tag_slug,
        'limit': limit,
        'active': 'true',
        'closed': 'false',
        'archived': 'false'
    }
    
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        # if the request is successful, we get the data
        if response.status_code == 200: # 200 means success, 400 means error, 500 means server error
            events_data = response.json()
            if events_data and isinstance(events_data, list): # if the output is a list
                
                slugs = [event.get('slug') for event in events_data if event.get('slug')] # get the slug for each event
                if specific_event:
                    slugs = [i for i in slugs if specific_event.lower() in i.lower()]
                print(f"   ✅ Found {len(slugs)} event slugs")
                return slugs
            else:
                print(f"   ❌ No events data returned")
                return []
        else: 
            print(f"   ❌ API Error: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"   ❌ Exception: {str(e)}")
        return []
    

### 2.2 Kalshi Functions <a id='2_2'></a>

**get_kalshi_events(series_ticker)**:
- Get Kalshi events data using the series_ticker parameter, by default we get MLB data
- "event_ticker" in the form 'KXMLBGAME-25JUL09ATLATH'
- 

**get_kalshi_markets_for_event(event_ticker)**:
- Get the Market data for that specific event (with event_ticker) and retrieve market_id to get orderbook
- It will return 2 items for binary contract (orderbooks for markets on both sides) or more for multi-outcome events

**get_kalshi_games_markets_df(series_ticker)**:
- Fetches all binary events, then builds a DataFrame from there
- First calls **get_kalshi_events(series_ticker)**
- Then calls **get_kalshi_markets_for_event(event_ticker)**

**get_kalshi_orderbook(market_id)**:
- Get the entire orderbook from a single Kalshi market
- Output has orderbook for "yes" and "no" price for that specific event ("Will Atlanta Braves win tonight?")
- The price from the orderbooks are the bid price

**get_kalshi_orderbook_arrays_with_tickers(ticker_a, ticker_b)**:
- Return the bid and ask prices for a single market.
- We return the orderbooks from both sides, since orderbooks are mirrored in multi outcome event.

**trade_limit_order_kalshi(action, ticker, price, count, side='yes')**:
- Action = 'buy' or 'sell'
- ticker is the market ticker
- price and count are obvious
- side = 'yes' or 'no'

**kalshi_to_pm(ks_ticker)**:
- Convert kalshi event_ticker to pm slug

In [110]:
def get_kalshi_events(series_ticker="KXMLBGAME"):
    """
    Get Kalshi events data using the series_ticker parameter
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/events"
    
    # Parameters for the API call
    params = {
        "status": "open", # Only get open events
        "series_ticker": series_ticker # MLB ticker by default
    }
    
    try:       
        response = requests.get(url, params=params)
        print(f"Status Code: {response.status_code}") # 200 means success, 400 means error, 500 means server error
        
        # if the request is successful, we get the data
        if response.status_code == 200:
            data = response.json() # convert the response to a JSON object
            print(f"Successfully retrieved data!") # print a success message
            print(f"Number of events: {len(data.get('events', []))}") # print the number of events
            return data
            
        else:
            print(f"Error: HTTP {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"Error making API call: {e}")
        return None

def get_kalshi_markets_for_event(event_ticker):
    """
    Get markets data for a specific event ticker
    """
    url = "https://api.elections.kalshi.com/trade-api/v2/markets"
    
    params = {
        "event_ticker": event_ticker
    }
    
    try:
        print(f"Getting markets for {event_ticker}")
        response = requests.get(url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            return data
        else:
            print(f"  Error: HTTP {response.status_code}")
            print(f"  Response: {response.text}")
            return None
            
    except Exception as e:
        print(f"  Error getting markets for {event_ticker}: {e}")
        return None

def get_kalshi_markets_df(series_ticker="KXMLBGAME"):
    """
    Fetches all baseball events, then builds a DataFrame where each row
    is a single market, annotated with its parent event_ticker.
    """
    # 1) pull your events
    events_data = get_kalshi_events(series_ticker)
    events = events_data.get('events', [])
    event_tickers = [e['event_ticker'] for e in events if e.get('event_ticker')]

    # 2) flatten markets into “records”
    records = []
    for ticker in event_tickers:
        markets_data = get_kalshi_markets_for_event(ticker)
        for m in markets_data.get('markets', []):
            # include any fields you want—here we take all market keys
            rec = {
                "event_ticker": ticker,
                **m
            }
            records.append(rec)
        time.sleep(0.1)

    # 3) build the DataFrame
    df = pd.DataFrame(records)
    return df
    
def get_kalshi_orderbook(market_id, depth=10):
    url = f"https://api.elections.kalshi.com/trade-api/v2/markets/{market_id}/orderbook"
    if depth:
        params = {"accept": "application/json",
                  "depth": depth}
    else:
        params = {"accept": "application/json"}

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: HTTP {response.status_code}")
        print(f"Response: {response.text}")
        return None
    
def get_kalshi_orderbook_arrays_with_tickers(ticker_a, ticker_b):
    # get orderbooks for ticker_a and ticker_b
    # ------------------------------
    # ticker_a orderbook
    # ------------------------------
    kalshi_ticker_a_orderbook = get_kalshi_orderbook(ticker_a, depth=10) 
    kalshi_orderbooks_a = [np.array(kalshi_ticker_a_orderbook.get("orderbook").get(option)) for option in ["yes", "no"]] # get orderbooks for yes and no

    # yes_bid_prices_a: bid price for yes, ask price for no
    kalshi_yes_bid_prices_a = kalshi_orderbooks_a[0]
    kalshi_no_ask_prices_a = kalshi_orderbooks_a[0].copy() # Mirror yes_bid_prices_a to get no_ask_prices_a
    kalshi_no_ask_prices_a[:, 0] = 100 - kalshi_no_ask_prices_a[:, 0] # subtract each value in the *first column* from 100 to get ask price for the other side

    # no_bid_prices_a: ask price for yes, bid price for no
    kalshi_no_bid_prices_a = kalshi_orderbooks_a[1]
    kalshi_yes_ask_prices_a = kalshi_orderbooks_a[1].copy() # Mirror
    kalshi_yes_ask_prices_a[:, 0] = 100 - kalshi_yes_ask_prices_a[:, 0] # subtract each value in the *first column* from 100 to get ask price for the other side


    # ------------------------------
    # ticker_b orderbook
    # ------------------------------
    kalshi_ticker_b_orderbook = get_kalshi_orderbook(ticker_b, depth=10) 
    kalshi_orderbooks_b = [np.array(kalshi_ticker_b_orderbook.get("orderbook").get(option)) for option in ["yes", "no"]] # get orderbooks for yes and no

    # yes_bid_prices_b: bid price for yes, ask price for no
    kalshi_yes_bid_prices_b = kalshi_orderbooks_b[0]
    kalshi_no_ask_prices_b = kalshi_orderbooks_b[0].copy() # Mirror yes_bid_prices_b to get no_ask_prices_b
    kalshi_no_ask_prices_b[:, 0] = 100 - kalshi_no_ask_prices_b[:, 0] # subtract each value in the *first column* from 100 to get ask price for the other side

    # no_bid_prices_b: ask price for yes, bid price for no
    kalshi_no_bid_prices_b = kalshi_orderbooks_b[1]
    kalshi_yes_ask_prices_b = kalshi_orderbooks_b[1].copy() # Mirror
    kalshi_yes_ask_prices_b[:, 0] = 100 - kalshi_yes_ask_prices_b[:, 0] # subtract each value in the *first column* from 100 to get ask price for the other side
    return kalshi_yes_ask_prices_a, kalshi_yes_bid_prices_a, kalshi_no_ask_prices_a, kalshi_no_bid_prices_a, kalshi_yes_ask_prices_b, kalshi_yes_bid_prices_b, kalshi_no_ask_prices_b, kalshi_no_bid_prices_b

def kalshi_to_pm(ks_ticker: str) -> str:
    """
    Convert a Kalshi ticker (e.g. "KXMLBGAME-25JUL10ARISD")
    back into a Polymarket slug ("mlb-ari-sd-2025-07-10").
    """
    # strip off the prefix and split
    _, body = ks_ticker.split('-', 1)                # "25JUL10ARISD"
    # parse the date
    dt = datetime.strptime(body[:7], "%y%b%d")       # first 7 chars
    iso_date = dt.strftime("%Y-%m-%d")               # "2025-07-10"
    team_codes = body[7:]                            # the rest, e.g. "ARISD"

    # now we need to figure out where to split team_codes back into t1 and t2.
    # Easiest is to look at your existing Polymarket slugs and match:
    for pm_slug in all_pm_slugs:  # you should have discovered these already
        if pm_slug.endswith(iso_date) and \
           team_codes == (pm_slug.split('-')[1].upper() + pm_slug.split('-')[2].upper()):
            return pm_slug

    raise ValueError(f"could not find matching PM slug for {ks_ticker}")

def trade_limit_order_kalshi(action, ticker, price, count, side='yes'):
    BASE     = "https://api.elections.kalshi.com"
    ENDPOINT = "/trade-api/v2/portfolio/orders"

    # 1) HEAD → get server time as an *aware* datetime
    head      = requests.head(BASE + ENDPOINT, timeout=5)
    server_dt = parsedate_to_datetime(head.headers["Date"])    # e.g. 23:06:58+00:00

    # 2) Build timestamp ms directly from that
    ts_ms = str(int(server_dt.timestamp() * 1000))             # correct UTC epoch

    # 3) Build & sign the preimage with RSA-PSS
    preimage = (ts_ms + "POST" + ENDPOINT).encode()
    pem      = open(os.environ["KALSHI_PRIVATE_KEY_PATH"], "rb").read()
    priv     = serialization.load_pem_private_key(pem, password=None)
    sig      = priv.sign(
        preimage,
        padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    signature_b64 = base64.b64encode(sig).decode()

    # 4) Assemble headers & body
    headers = {
    "KALSHI-ACCESS-KEY":       os.getenv("KALSHI_ACCESS_KEY"),
    "KALSHI-ACCESS-TIMESTAMP": ts_ms,
    "KALSHI-ACCESS-SIGNATURE": signature_b64,
    "Content-Type":            "application/json",
    }

    # Kalshi has to specify "yes_price" or "no_price"
    side_price = "yes_price" if side == "yes" else "no_price"
    body = {
    "action":          action, # buy or sell
    "client_order_id": str(uuid.uuid4()), # unique order id
    "count":           count, # number of contracts
    "side":            side, # yes or no
    "ticker":          ticker, # ticker of the contract
    "type":            "limit", # limit order to match "maker"
    side_price:        price # price of the contract
    # They also have extra field like :
    #   "post_only": If this flag is set to true, an order will be rejected if it crosses the spread and executes.
    #   "sell_position_floor": SellPositionFloor will not let you flip position for a market order if set to 0.
    #   "time_in_force": Only "fill_or_kill"is supported. Other time in forces are controlled through expiration_ts
    #   "expiration_ts": Expiration time of the order, in unix seconds. Use for Good 'Till Cancelled (GTC).
    }

    # 5) Fire off the order
    resp = requests.post(BASE + ENDPOINT, headers=headers, json=body, timeout=10)
    print(resp.status_code, resp.json())
    return


## 3. Collect Data <a id='3'></a>

### 3.1 Collect Polymarket Data <a id='3_1'></a>

In [111]:
pm_data = get_polymarket_games_markets_df()

⚾ POLYMARKET MLB LIVE MARKETS (Date-Filtered Discovery)
🕐 2025-07-13 14:21:57
🔗 Using date-filtered events API discovery

🔍 DISCOVERING EVENTS: 2025-07-12 to 2025-07-13
🔍 Discovering slugs from 2025-07-12 to 2025-07-13...
   ✅ Found 15 event slugs
   🔍 Getting market data for mlb-mia-bal-2025-07-13
   🔍 Getting market data for mlb-tb-bos-2025-07-13
   🔍 Getting market data for mlb-chc-nyy-2025-07-13
   🔍 Getting market data for mlb-sea-det-2025-07-13
   🔍 Getting market data for mlb-col-cin-2025-07-13
   🔍 Getting market data for mlb-tex-hou-2025-07-13
   🔍 Getting market data for mlb-cle-cws-2025-07-13
   🔍 Getting market data for mlb-pit-min-2025-07-13
   🔍 Getting market data for mlb-nym-kc-2025-07-13
   🔍 Getting market data for mlb-wsh-mil-2025-07-13
   🔍 Getting market data for mlb-atl-stl-2025-07-13
   🔍 Getting market data for mlb-lad-sf-2025-07-13
   🔍 Getting market data for mlb-tor-oak-2025-07-13
   🔍 Getting market data for mlb-ari-laa-2025-07-13
   🔍 Getting market data fo

#### 3.1.1 Collect Polymarket Orderbook <a id='3_1_1'></a>

In [112]:
pm_data.iloc[0]

slug                                                     mlb-mia-bal-2025-07-13
market_id                                                                562851
condition_id                  0x1b7e52ecb79d1f890260961149708095668ae9f6a794...
question_id                   0xab6aca3731ae1b9a39c68fb2cf72b1b2610f2e307232...
title                                                       Marlins vs. Orioles
game_time                                                2025-07-13 17:35:00+00
description                   In the upcoming MLB game, scheduled for July 1...
resolution_source                                          https://www.mlb.com/
active                                                                     True
closed                                                                    False
accepting_orders                                                           True
accepting_orders_timestamp                                 2025-07-12T08:00:34Z
new                                     

In [113]:
pm_ticker_a = pm_data.loc[14, 'clob_token_ids'][0]
pm_ticker_b = pm_data.loc[0, 'clob_token_ids'][1]
pm_a_asks, pm_a_bids, pm_b_asks, pm_b_bids = get_polymarket_orderbook_array_with_tickers(pm_ticker_a, pm_ticker_b)

### 3.2 Collect Kalshi Data <a id='3_2'></a>

In [114]:
# Get the market ticker, event id, and basic orderbook data
kalshi_data = get_kalshi_markets_df()

Status Code: 200
Successfully retrieved data!
Number of events: 18
Getting markets for KXMLBGAME-25JUL13PHISD
Getting markets for KXMLBGAME-25JUL13ARILAA
Getting markets for KXMLBGAME-25JUL13TORATH
Getting markets for KXMLBGAME-25JUL13LADSF
Getting markets for KXMLBGAME-25JUL13ATLSTL
Getting markets for KXMLBGAME-25JUL13WSHMIL
Getting markets for KXMLBGAME-25JUL13TEXHOU
Getting markets for KXMLBGAME-25JUL13PITMIN
Getting markets for KXMLBGAME-25JUL13NYMKC
Getting markets for KXMLBGAME-25JUL13CLECWS
Getting markets for KXMLBGAME-25JUL13SEADET
Getting markets for KXMLBGAME-25JUL13COLCIN
Getting markets for KXMLBGAME-25JUL13TBBOS
Getting markets for KXMLBGAME-25JUL13MIABAL
Getting markets for KXMLBGAME-25JUL13CHCNYY
Getting markets for KXMLBGAME-25JUN18MILCHC
Getting markets for KXMLBGAME-25MAY21ATLWSH
Getting markets for KXMLBGAME-25MAY20CLEMIN


#### 3.2.1. Collect Kalshi Orderbook <a id='3_2_1'></a>

In [115]:
# get orderbooks for ticker_a and ticker_b
ticker_a = kalshi_data.iloc[0, 1]
ticker_b = kalshi_data.iloc[1, 1]
k_a_yes_ask, k_a_yes_bid, k_a_no_ask, k_a_no_bid, k_b_yes_ask, k_b_yes_bid, k_b_no_ask, k_b_no_bid = get_kalshi_orderbook_arrays_with_tickers(ticker_a, ticker_b)

In [116]:
discover_future_slugs()

🔍 Discovering slugs for mlb
   ✅ Found 50 event slugs


['world-series-champion-2025',
 'american-league-winner-2025',
 'national-league-winner-2025',
 'mlb-al-mvp',
 'mlb-nl-mvp',
 'mlb-al-cy-young',
 'mlb-nl-cy-young',
 'al-east-division-winner',
 'al-central-division-winner',
 'al-west-division-winner',
 'nl-east-division-winner',
 'nl-central-division-winner',
 'nl-west-division-winner',
 'mlb-home-run-leader',
 'mlb-steals-leader',
 'mlb-rbi-leader',
 'will-the-yankees-set-the-single-season-hr-record',
 'will-aaron-judge-break-hr-record',
 'mlb-atl-mia-2025-04-06',
 'mlb-tor-bal-2025-04-11',
 'al-rookie-of-the-year',
 'nl-rookie-of-the-year',
 'will-aaron-judge-win-triple-crown',
 'mlb-best-regular-season-record',
 'mlb-worst-regular-season-record',
 'mlb-cle-min-2025-05-20',
 'mlb-atl-wsh-2025-05-21',
 'mlb-pitching-era-leader',
 'mlb-pitching-strikeouts-leader',
 'mlb-mil-chc-2025-06-18',
 'mlb-which-teams-make-the-playoffs',
 'mlb-all-star-game-al-starting-pitcher',
 'mlb-all-star-game-nl-starting-pitcher',
 'mlb-all-star-game-mvp',

### 3.3 Compare PM and Kalshi Data for One Market <a id='3_3'></a>

#### 3.3.1 Match Slugs and Tickers <a id='3_3_1'></a>

In [25]:
all_slugs = pm_data['slug'].tolist()

# 1) Build the forward map: PM slug → Kalshi ticker
mapping_pm2ks = {
    slug: pm_to_kalshi(slug)
    for slug in pm_data['slug']
}

# 2) Add it into your Polymarket frame
pm_data['event_ticker'] = pm_data['slug'].map(mapping_pm2ks)

# 3) Build the reverse map: Kalshi ticker → PM slug
mapping_ks2pm = { v: k for k, v in mapping_pm2ks.items() }

# 4) Add it into your Kalshi frame
kalshi_data['pm_slug'] = kalshi_data['event_ticker'].map(mapping_ks2pm)
kalshi_data_clean = kalshi_data.dropna(subset=['pm_slug'])
pm_data_clean = pm_data[pm_data['slug'].isin(kalshi_data_clean['pm_slug'].to_list())]

pm_data_clean.head()

,slug,market_id,condition_id,question_id,title,game_time,description,resolution_source,active,closed,...,updated_at,fee,fpmm_live,neg_risk,rfq_enabled,approved,funded,outcomes,prices,event_ticker
0,mlb-mia-bal-2025-07-13,562851,0x1b7e52ecb79d1f890260961149708095668ae9f6a794...,0xab6aca3731ae1b9a39c68fb2cf72b1b2610f2e307232...,Marlins vs. Orioles,2025-07-13 17:35:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:51:54.031046Z,20000000000000000,True,False,False,True,False,"[Marlins, Orioles]","[0.435, 0.565]",KXMLBGAME-25JUL13MIABAL
1,mlb-tb-bos-2025-07-13,562852,0x7fb9bf16b55aeeaee19c905908a79ce6e753eb665a15...,0x60251fd6ee0d333a5214109e423b055cb793724aa9de...,Rays vs. Red Sox,2025-07-13 17:35:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:51:03.667864Z,20000000000000000,True,False,False,True,False,"[Rays, Red Sox]","[0.475, 0.525]",KXMLBGAME-25JUL13TBBOS
2,mlb-chc-nyy-2025-07-13,562853,0x2f01ba8a397f2b7e3a445811141fa861074233ea6344...,0x83a688755e743bc269fabea54e81abed201e5417c03f...,Cubs vs. Yankees,2025-07-13 17:35:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:50:26.658371Z,20000000000000000,True,False,False,True,False,"[Cubs, Yankees]","[0.465, 0.535]",KXMLBGAME-25JUL13CHCNYY
3,mlb-sea-det-2025-07-13,562854,0xdbe3b8c2836f8523e94fd61b3d1952a7f0bf319a42f0...,0x56b0161dcd67adb96b3ba26bf62d436060c070466f51...,Mariners vs. Tigers,2025-07-13 17:40:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:55:06.283488Z,20000000000000000,True,False,False,True,False,"[Mariners, Tigers]","[0.495, 0.505]",KXMLBGAME-25JUL13SEADET
4,mlb-col-cin-2025-07-13,562855,0x8126ee72a8093f27bbe45c5c381aeafc5603045d3f3d...,0x62db125aa7194c7aafbe841cf887b18c5a4db07847b8...,Rockies vs. Reds,2025-07-13 17:40:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:53:13.205494Z,20000000000000000,True,False,False,True,False,"[Rockies, Reds]","[0.3, 0.7]",KXMLBGAME-25JUL13COLCIN


#### 3.3.2 Trade on Polymarket <a id='3_3_2'></a>

In [30]:
# Get the tickers for the Dodgers vs. Padres game on Polymarket
pm_ticker_a = pm_data.loc[11, 'clob_token_ids'][0]
pm_ticker_b = pm_data.loc[11, 'clob_token_ids'][1]

pm_a_asks, pm_a_bids, pm_b_asks, pm_b_bids = get_polymarket_orderbook_array_with_tickers(pm_ticker_a, pm_ticker_b)

# Dodgers vs. Padres. a is Dodgers, b is Padres
pm_a_asks

# Get the tickers for the Dodgers vs. Padres game on Kalshi
ticker_a = kalshi_data.iloc[0, 1]
ticker_b = kalshi_data.iloc[1, 1]
k_a_yes_ask, k_a_yes_bid, k_a_no_ask, k_a_no_bid, k_b_yes_ask, k_b_yes_bid, k_b_no_ask, k_b_no_bid = get_kalshi_orderbook_arrays_with_tickers(ticker_a, ticker_b)

{'market': '0xb81c016e04d627c145cc5b240af45a36928321eeb62b00b7e554af85634931bb', 'asset_id': '47694126269003497736767825546397900273942213538017141506653369691554553660298', 'timestamp': '1752425888243', 'hash': 'd75042aabad76ddeaa418faf533a7d0e5e19951c', 'bids': [{'price': '0.01', 'size': '150'}, {'price': '0.25', 'size': '30000'}, {'price': '0.26', 'size': '3000'}, {'price': '0.49', 'size': '24004'}, {'price': '0.51', 'size': '39.2'}, {'price': '0.52', 'size': '10000'}, {'price': '0.53', 'size': '30095'}, {'price': '0.54', 'size': '11818'}, {'price': '0.55', 'size': '35676.43'}, {'price': '0.56', 'size': '18726.88'}], 'asks': [{'price': '0.75', 'size': '30000'}, {'price': '0.74', 'size': '3000'}, {'price': '0.65', 'size': '26493'}, {'price': '0.64', 'size': '27.77'}, {'price': '0.62', 'size': '10000'}, {'price': '0.61', 'size': '31115.64'}, {'price': '0.6', 'size': '11006'}, {'price': '0.59', 'size': '21319.95'}, {'price': '0.58', 'size': '23840.33'}]}
Time taken to get orderbook: 0.

array([[   75, 30000],
       [   74,  3000],
       [   65, 26493],
       [   64,    27],
       [   62, 10000],
       [   61, 31115],
       [   60, 11006],
       [   59, 21319],
       [   58, 23840]])

In [ ]:
'mlb-ari-laa-2025-07-13'
pm_ticker_a = pm_data.loc[11, 'clob_token_ids'][0]
pm_ticker_b = pm_data.loc[0, 'clob_token_ids'][1]
pm_a_asks, pm_a_bids, pm_b_asks, pm_b_bids = get_polymarket_orderbook_array_with_tickers(pm_ticker_a, pm_ticker_b)

In [26]:
pm_data_clean.tail()

,slug,market_id,condition_id,question_id,title,game_time,description,resolution_source,active,closed,...,updated_at,fee,fpmm_live,neg_risk,rfq_enabled,approved,funded,outcomes,prices,event_ticker
9,mlb-wsh-mil-2025-07-13,562860,0x7177853c5d6e29ccdd6717feabfdf2205c8a82125292...,0x682454090400979b2c5778ef20cd943b70a416c36b9b...,Nationals vs. Brewers,2025-07-13 18:10:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:53:05.789776Z,20000000000000000,True,False,False,True,False,"[Nationals, Brewers]","[0.285, 0.715]",KXMLBGAME-25JUL13WSHMIL
10,mlb-atl-stl-2025-07-13,562861,0xef9aa264191b33532f80fc03ea3ab1c12248bd95ad83...,0xdc533802e1a0d774a4d8772a991d202e4bea29d489a3...,Braves vs. Cardinals,2025-07-13 18:15:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:55:05.635677Z,20000000000000000,True,False,False,True,False,"[Braves, Cardinals]","[0.395, 0.605]",KXMLBGAME-25JUL13ATLSTL
11,mlb-lad-sf-2025-07-13,562862,0xb81c016e04d627c145cc5b240af45a36928321eeb62b...,0x26972a1528314087b986adc0621233b6b03092f7ac57...,Dodgers vs. Giants,2025-07-13 20:05:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:51:09.126656Z,20000000000000000,True,False,False,True,False,"[Dodgers, Giants]","[0.565, 0.435]",KXMLBGAME-25JUL13LADSF
13,mlb-ari-laa-2025-07-13,562864,0x423a2e2c2838e0f151d8478866ce1b212b3dc753b1c8...,0x8bc7d675f9507bc3681040767ff8cf7140ff6536d9a1...,Diamondbacks vs. Angels,2025-07-13 20:07:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:50:31.410609Z,20000000000000000,True,False,False,True,False,"[Diamondbacks, Angels]","[0.485, 0.515]",KXMLBGAME-25JUL13ARILAA
14,mlb-phi-sd-2025-07-13,562865,0x9db689090a8ad765376d0e348953edeae11d8dff029e...,0x87c0160a9e299754d054e430a027af8af13d8af3ed9a...,Phillies vs. Padres,2025-07-13 20:10:00+00,"In the upcoming MLB game, scheduled for July 1...",https://www.mlb.com/,True,False,...,2025-07-13T16:50:34.165691Z,20000000000000000,True,False,False,True,False,"[Phillies, Padres]","[0.545, 0.455]",KXMLBGAME-25JUL13PHISD


#### 3.3.3 Trade on Kalshi <a id='3_3_3'></a>

In [126]:
transaction_fee(100, 0.5, True)

1.76

In [168]:
def transaction_fee(contract_size, price, taker=True):
    # Can be optimized
    return math.ceil((0.07 if taker else 0.0175) * contract_size * price * (1 - price) * 100) / 100

contract_size = 150
kalshi_price = 0.41
taker = False

kalshi_tran_fee = transaction_fee(contract_size, kalshi_price, taker)
kalshi_total_price = contract_size * kalshi_price + kalshi_tran_fee

polymarket_price = 0.57
polymarket_total_price = contract_size * polymarket_price + 0.05

print(f"Contract size is ${contract_size}")

print(f"Kalshi price is ${kalshi_price} * ${contract_size} + ${kalshi_tran_fee}(Transaction fee) = ${int(kalshi_total_price * 1000)/1000}")
print(f"Polymarket price is ${polymarket_price} * ${contract_size} + ${0.05}(Transaction Fee) = ${int(polymarket_total_price * 1000)/1000}")
print(f"Final payoff is ${contract_size} - ${kalshi_total_price} - ${polymarket_total_price} = ${int(1000 * (contract_size - kalshi_total_price - polymarket_total_price))/1000}")
print(f"Profit rate: {int(10000 *(contract_size - kalshi_total_price - polymarket_total_price)/(polymarket_total_price + kalshi_total_price))/100}%") 

Contract size is $150
Kalshi price is $0.41 * $150 + $0.64(Transaction fee) = $62.139
Polymarket price is $0.57 * $150 + $0.05(Transaction Fee) = $85.549
Final payoff is $150 - $62.13999999999999 - $85.54999999999998 = $2.31
Profit rate: 1.56%


In [170]:
def return_bet_amount(x, price, taker=True):
    return x/price - x

In [190]:
116 * 0.67

77.72

In [197]:
contract_size = 70
p = 0.67

print(f'{return_bet_amount(contract_size, p)} - {transaction_fee(contract_size, p)} = {return_bet_amount(contract_size, p) - transaction_fee(contract_size, p)}')

34.4776119402985 - 1.09 = 33.3876119402985


In [243]:
363 * 0.67

243.21

In [245]:
253 * 0.7 + transaction_fee(253, 0.7, True)

180.82

In [246]:
(253 * 0.7 + transaction_fee(253, 0.7, True))/0.98 + 1

185.51020408163265

In [ ]:
contract_size_sharma = 568
price_sharma = 0.67
sharma_amount = contract_size_sharma * price_sharma
sharma_amount_with_fee = sharma_amount + transaction_fee(contract_size_sharma, price_sharma)
print(f"Total Amount on Sharma: {contract_size_sharma} * ${price_sharma} + ${transaction_fee(contract_size_sharma, price_sharma)}(Transaction Fee) = ${sharma_amount}")
print(f"Total Amount on Sharma with fee: ${sharma_amount} + ${transaction_fee(contract_size_sharma, price_sharma)} = ${sharma_amount_with_fee}")

contract_size_jac = 813
price_jac = 0.06
jacquemot_amount = contract_size_jac * price_jac
jacquemot_amount_with_fee = jacquemot_amount + transaction_fee(contract_size_jac, price_jac)
print()
print(f"Total Amount on Jacquemot: {contract_size_jac} * ${price_jac} + ${transaction_fee(contract_size_jac, price_jac)}(Transaction Fee) = ${jacquemot_amount}")
print(f"Total Amount on Jacquemot with fee: ${jacquemot_amount} + ${transaction_fee(contract_size_jac, price_jac)} = ${jacquemot_amount_with_fee}")
print()
print(f"Total Amount Betted Pre Deposit Fee: ${sharma_amount_with_fee} + ${jacquemot_amount_with_fee} = ${sharma_amount_with_fee + jacquemot_amount_with_fee}")

print()
number_of_deposit = 4
total_deposit_amount = 150 + 250
total_after_processing_fee = total_deposit_amount - number_of_deposit
transaction_fees_processing = total_after_processing_fee * 0.02 + number_of_deposit

print(f"Total Transaction Fee: ${number_of_deposit} + ${total_after_processing_fee} * 2% = ${transaction_fees_processing}")

total_amount = sharma_amount_with_fee + jacquemot_amount_with_fee + transaction_fees_processing
print(f"Total Amount: ${total_amount}")
print()
print(f"Case (1): if Sharma wins, we win ${contract_size_sharma} - ${total_amount} = ${contract_size_sharma - total_amount}")
print(f"Case (2): if Jacquemot wins, we win ${contract_size_jac} - ${total_amount} = ${contract_size_jac - total_amount}")

Total Amount on Sharma: 568 * $0.67 + $8.8(Transaction Fee) = $380.56
Total Amount on Sharma with fee: $380.56 + $8.8 = $389.36

Total Amount on Jacquemot: 813 * $0.06 + $3.21(Transaction Fee) = $48.78
Total Amount on Jacquemot with fee: $48.78 + $3.21 = $51.99

Total Amount Betted Pre Deposit Fee: $389.36 + $51.99 = $441.35

Total Transaction Fee: $4 + $396 * 2% = $11.92
Total Amount: $453.27000000000004

Case (1): if Sharma wins, we win $568 - $453.27000000000004 = $114.72999999999996
Case (2): if Jacquemot wins, we win $813 - $453.27000000000004 = $359.72999999999996


In [185]:
kalshi_first = (0.06 * 813 + transaction_fee(813, 0.06, True))
kalshi_second = (0.67 * 143 + transaction_fee(143, 0.67, True))
with_transaction_fee = kalshi_second + kalshi_first + 2 + 98 * 0.02

print(f"Kalshi first: ${kalshi_first}")
print(f"Kalshi second: ${kalshi_second}")
print(f"Total: ${kalshi_second + kalshi_first} + $3 + $49 * 0.02 = ${kalshi_second + kalshi_first + 2 + 98 * 0.02}")

Kalshi first: $51.99
Kalshi second: $98.03
Total: $150.02 + $3 + $49 * 0.02 = $153.98000000000002


In [186]:
with_transaction_fee

153.98000000000002

In [182]:
transaction_fee(143, 0.67, True)

2.22

In [183]:
transaction_fee(813, 0.06, True)

3.21

### 3.4 Identify Arbitrage and Trade <a id='3_4'></a>

#### 3.4.1 Specify Tickers and Trade <a id='3_4_1'></a>

In [122]:
# ---------------------------------------POLYMARKET---------------------------------------
# Get the tickers for the Dodgers vs. Giants game on Polymarket
index = 11
a_acr = "LAD"
b_acr = "-SF"
a_name = "Dodgers"
b_name = "Giants"

pm_ticker_a = pm_data.loc[index, 'clob_token_ids'][0]
pm_name_a =  pm_data.loc[index, "outcomes"][0]
pm_name_a = a_acr if pm_name_a == a_name else b_acr
pm_ticker_b = pm_data.loc[index, 'clob_token_ids'][1]
pm_name_b =  pm_data.loc[index, "outcomes"][1]
pm_name_b = a_acr if pm_name_b == a_name else b_acr

# ---------------------------------------KALSHI---------------------------------------

# Get the tickers for the Dodgers vs. Padres game on KALSHI
corresponding_kalshi_data = kalshi_data_clean[kalshi_data_clean['pm_slug'] == pm_data.loc[index, 'slug']]

kalshi_ticker_a = corresponding_kalshi_data.iloc[0, 1] if corresponding_kalshi_data.iloc[0, 1][-3:] == pm_name_a else corresponding_kalshi_data.iloc[1, 1]
kalshi_ticker_b = corresponding_kalshi_data.iloc[0, 1] if corresponding_kalshi_data.iloc[0, 1][-3:] == pm_name_b else corresponding_kalshi_data.iloc[1, 1]

In [136]:
def arbitrage_kalshi_pm(contract_size, kalshi_price, kalshi_taker, polymarket_price):
    # Calculate the transaction fee for Kalshi
    kalshi_cost = contract_size * kalshi_price
    kalshi_tran_fee = transaction_fee(contract_size, kalshi_price, kalshi_taker) # Calculate the transaction fee for Kalshi
    kalshi_total_price = kalshi_cost + kalshi_tran_fee

    polymarket_cost = contract_size * polymarket_price
    polymarket_tran_fee = 0.05 # TODO: Add polymarket transaction fee
    polymarket_total_price = polymarket_cost + polymarket_tran_fee
    profit = contract_size - kalshi_total_price - polymarket_total_price
    if (profit < 0):
        return profit
    print(f"Contract size is {contract_size}")
    print(f"Kalshi price is ${kalshi_price} * {contract_size} + ${kalshi_tran_fee}(Transaction fee) = ${kalshi_total_price=}")
    print(f"Polymarket price is ${polymarket_price} * {contract_size} + ${0.05}(Transaction Fee) = ${polymarket_total_price}")
    print(f"Final payoff is ${contract_size} - ${kalshi_total_price} - ${polymarket_total_price} = ${profit}")
    print("--------------------------------")
    return profit

In [138]:
total_profit = 0
i = 0
print(f"Starting arbitrage for {a_name} vs. {b_name}")
profits = []
times = []

while True:
    i += 1
    # Get the orderbook for the Polymarket and Kalshi markets
    if i % 100 == 0:
        print(f"Total profit: {total_profit}")
    pm_a_asks, pm_a_bids, pm_b_asks, pm_b_bids = get_polymarket_orderbook_array_with_tickers(pm_ticker_a, pm_ticker_b)
    k_a_yes_ask, k_a_yes_bid, k_a_no_ask, k_a_no_bid, k_b_yes_ask, k_b_yes_bid, k_b_no_ask, k_b_no_bid = get_kalshi_orderbook_arrays_with_tickers(kalshi_ticker_a, kalshi_ticker_b)

    # Polymarket orderbook for Dodgers vs. Giants
    # PM_A_ASKS:                  k_a_no_ask
    # [60,  30794],               [48, 10000],
    # [59,  12718],               [47, 10000],
    # [58, 125175],               [46, 22434],
    # [57,  63928]                [45,  4682]

    if (pm_a_asks[-1][0] + k_a_no_ask[-1][0] < 100):
        print(f"Potential Arbitrage opportunity {i}(1) found: Polymarket A Wins and Kalshi A Loses")
        # Get the contract size by getting the minimum of the last ask price of Polymarket and Kalshi
        contract_size = min(min(pm_a_asks[-1][1], k_a_no_ask[-1][1]), 100)
        kalshi_price = k_a_no_ask[-1][0] / 100
        polymarket_price = pm_a_asks[-1][0] / 100
        profit = arbitrage_kalshi_pm(contract_size, kalshi_price, True, polymarket_price)
        if (profit > 0):
            total_profit += profit
            profits.append(profit)
            times.append(time.time())
            print()
    elif (pm_a_asks[-1][0] + k_b_yes_ask[-1][0] < 100):
        print(f"Potential Arbitrage opportunity {i}(2) found: Polymarket A Wins and Kalshi B Wins")
        # Get the contract size by getting the minimum of the last ask price of Polymarket and Kalshi
        contract_size = min(min(pm_a_asks[-1][1], k_b_yes_ask[-1][1]), 100)
        kalshi_price = k_b_yes_ask[-1][0] / 100
        polymarket_price = pm_a_asks[-1][0] / 100
        profit = arbitrage_kalshi_pm(contract_size, kalshi_price, True, polymarket_price)
        if (profit > 0):
            total_profit += profit
            profits.append(profit)
            times.append(time.time())
            print()
    elif (pm_b_asks[-1][0] + k_b_no_ask[-1][0] < 100):
        print(f"Potential Arbitrage opportunity {i}(3) found: Polymarket B Wins and Kalshi B Loses")
        # Get the contract size by getting the minimum of the last ask price of Polymarket and Kalshi
        contract_size = min(min(pm_b_asks[-1][1], k_b_no_ask[-1][1]), 100)
        kalshi_price = k_b_no_ask[-1][0] / 100
        polymarket_price = pm_b_asks[-1][0] / 100
        profit = arbitrage_kalshi_pm(contract_size, kalshi_price, True, polymarket_price)
        if (profit > 0):
            total_profit += profit
            profits.append(profit)
            times.append(time.time())
            print()
    elif (pm_b_asks[-1][0] + k_a_yes_ask[-1][0] < 100):
        print(f"Potential Arbitrage opportunity {i}(4) found: Polymarket B Wins and Kalshi A Wins")
        # Get the contract size by getting the minimum of the last ask price of Polymarket and Kalshi
        contract_size = min(min(pm_b_asks[-1][1], k_a_yes_ask[-1][1]), 100)
        kalshi_price = k_a_yes_ask[-1][0] / 100
        polymarket_price = pm_b_asks[-1][0] / 100
        profit = arbitrage_kalshi_pm(contract_size, kalshi_price, True, polymarket_price)
        if (profit > 0):
            total_profit += profit
            profits.append(profit)
            times.append(time.time())
            print()
    else:
        continue

Starting arbitrage for Dodgers vs. Giants
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0
Total profit: 0


KeyboardInterrupt: 

In [65]:
k_a_yes_ask

array([[   75,   100],
       [   73,   100],
       [   72, 30054],
       [   66,     1],
       [   65,    47],
       [   61,  5000],
       [   60, 10750],
       [   59, 23900],
       [   58, 12048],
       [   57, 14787]])

In [59]:
corresponding_kalshi_data.iloc[0, 1]

'KXMLBGAME-25JUL13LADSF-SF'

In [41]:
kalshi_data_clean[kalshi_data_clean['pm_slug'] == pm_data.loc[11, 'slug']].iloc[1].ticker

'KXMLBGAME-25JUL13LADSF-LAD'

In [38]:
kalshi_data_clean[kalshi_data_clean['pm_slug'] == pm_data.loc[11, 'slug']]

,event_ticker,ticker,market_type,title,subtitle,yes_sub_title,no_sub_title,open_time,close_time,expected_expiration_time,...,result,can_close_early,expiration_value,category,risk_limit_cents,strike_type,custom_strike,rules_primary,rules_secondary,pm_slug
6,KXMLBGAME-25JUL13LADSF,KXMLBGAME-25JUL13LADSF-SF,binary,Los Angeles D at San Francisco Winner?,,San Francisco,San Francisco,2025-07-13T12:00:00Z,2027-07-13T20:05:00Z,2025-07-13T23:05:00Z,...,,True,,,0,structured,{'baseball_team': 'cef5e9c6-0724-4524-bdd1-000...,If San Francisco wins the Los Angeles D at San...,The following market refers to the Los Angeles...,mlb-lad-sf-2025-07-13
7,KXMLBGAME-25JUL13LADSF,KXMLBGAME-25JUL13LADSF-LAD,binary,Los Angeles D at San Francisco Winner?,,Los Angeles D,Los Angeles D,2025-07-13T12:00:00Z,2027-07-13T20:05:00Z,2025-07-13T23:05:00Z,...,,True,,,0,structured,{'baseball_team': '5ee3dc32-f192-4b80-ac4a-fd2...,If Los Angeles D wins the Los Angeles D at San...,The following market refers to the Los Angeles...,mlb-lad-sf-2025-07-13


In [192]:
from nba_api.stats.endpoints import ShotChartDetail
from nba_api.stats.static import players
from nba_api.stats.endpoints import PlayerProfileV2

# 1. Look up Kawhi’s player_id
player = players.find_players_by_full_name("Kawhi Leonard")[0]
pid = player["id"]
# 1. Pull every shot Kawhi took in 2024-25 RS
sc = ShotChartDetail(
    team_id=0,               # 0 = all teams
    player_id=pid,
    season_nullable="2024-25",
    season_type_all_star="Regular Season",
    context_measure_simple="FGA"   # returns location + FGM/FGA flags
)

df_shots = sc.get_data_frames()[0]
df_shots


Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')
